In [1]:
import requests
import bs4
import pandas as pd

In [2]:
def test(url):
    car_url=requests.get(url)
    while str(car_url) == '<Response [503]>':
        url = next_page(url)
        car_url=requests.get(url)
    return car_url

In [3]:
def scrap_rate2(url):
    #car_url=requests.get(url)
    car_url = test(url)
    car_soup = bs4.BeautifulSoup(car_url.text,'lxml')
    #get rate
    rate_all = car_soup.select('article cars-star-rating')
    stars=[i.text[0] for i in rate_all[0::7]]
    return stars

In [4]:
def scrap_rate(url):
    #get rate
    rate_all = url.select('article cars-star-rating')
    stars=[i.text[0] for i in rate_all[0::7]]
    return stars

In [5]:
def scrap_review(url):
    #get review
    review = url.select('p.review-card-text')
    review=[i.text.rstrip()[2:-2].lstrip() for i in review]
    return review

In [6]:
from datetime import datetime
def scrap_reviewTime(url):
    #get review_time
    review_time = url.select('p.review-card-review-by')
    review_time=[i.text.split('\n')[-2].lstrip()[3:] for i in review_time]
    review_time=[datetime.strptime(i, '%B %d, %Y') for i in review_time]
    return review_time

In [7]:
def scrap_reviewArea(url):
    #get area
    review_city = url.select('p.review-card-review-by')
    review_city = [i.text.split('\n')[-3].lstrip()[5:] for i in review_city]
    return review_city

In [14]:
def next_page(car_soup):
    #get tag
    links = car_soup.find_all("a", class_="cui-button cui-button--secondary")
    #print(links)
    page = links[0].get('href')[-7]
    #print(page)
    if len(links)==1 and page != '2':
        # Last Page
        return 'This is the last page'
    elif len(links)==1 and page == '2':
        # The next page of the first page
        next_links=links[0].get('href')
        next_links='https://www.cars.com'+next_links
        return next_links
    else:
        # Normal next page
        next_links=links[1].get('href')
        next_links='https://www.cars.com'+next_links
        return next_links

In [15]:
def scrap(make, model, year=['2018']):
    rate = []
    review = []
    reviewTime = []
    reviewArea = []
    for i in year:
        url='https://www.cars.com/research/'+make+'-'+model+'-'+i+'/consumer-reviews/'
        car_url=requests.get(url)
        car_soup = bs4.BeautifulSoup(car_url.text,'lxml')
        print('first page '+url)
        
        rate.extend(scrap_rate(car_soup))
        review.extend(scrap_review(car_soup))
        reviewTime.extend(scrap_reviewTime(car_soup))
        reviewArea.extend(scrap_reviewArea(car_soup))
        #print(rate)
        next_url = next_page(car_soup)
        print('second page '+next_url)
        while next_url != 'This is the last page':
            print('1 '+next_url)
            car_next_url=requests.get(next_url)
            car_next_soup = bs4.BeautifulSoup(car_next_url.text,'lxml')
            
            rate.extend(scrap_rate(car_next_soup))
            review.extend(scrap_review(car_next_soup))
            reviewTime.extend(scrap_reviewTime(car_next_soup))
            reviewArea.extend(scrap_reviewArea(car_next_soup))
            
            next_url = next_page(car_next_soup)
            print('2 '+next_url)
    print(len(rate))
    outcomes = pd.DataFrame({'Rate':rate,'Review':review, 'Time':reviewTime, 'Area':reviewArea})
    return outcomes